In [1]:
import oss2
import json
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
with open("../aliyun_key.json" ,'r') as key:
    AccessKeyDict=json.loads(key.read())
AccessKeyId=AccessKeyDict["AccessKeyId"]
AccessKeySecret=AccessKeyDict["AccessKeySecret"]
auth = oss2.Auth(AccessKeyId, AccessKeySecret)
bucket = oss2.Bucket(auth, 'https://oss-cn-shanghai.aliyuncs.com', 'ocr-yjs')
from aliyunsdkcore.client import AcsClient
from aliyunsdkcore.acs_exception.exceptions import ClientException
from aliyunsdkcore.acs_exception.exceptions import ServerException
from aliyunsdkcore.auth.credentials import AccessKeyCredential
from aliyunsdkcore.auth.credentials import StsTokenCredential
credentials = AccessKeyCredential(AccessKeyId, AccessKeySecret)
client = AcsClient(region_id='cn-shanghai', credential=credentials)
import seaborn as sns

In [3]:
"""
https://help.aliyun.com/document_detail/151968.htm
人脸属性识别
人脸属性识别可以识别检测人脸的性别、年龄、表情、眼镜、帽子五种属性，
返回人脸的1024维深度学习特征。基于这个特征并按照特征比较规则，可以实现高性能的人脸识别。
"""

from aliyunsdkfacebody.request.v20191230.RecognizeFaceRequest import RecognizeFaceRequest

request = RecognizeFaceRequest()
request.set_accept_format('json')

filepath="images/test2.jpg"
with open(filepath, 'rb') as fileobj:
    bucket.put_object(filepath, fileobj)
url="https://ocr-yjs.oss-cn-shanghai.aliyuncs.com/"+filepath
request.set_ImageURL(url)

response = client.do_action_with_exception(request)
response=json.loads(str(response, encoding='utf-8'))
print(response)

{'RequestId': '2CFA6578-1DDC-5845-ABC3-C39FF48CDF05', 'Data': {'BeautyList': [89], 'FaceRectangles': [75, 88, 133, 173], 'Qualities': {'ScoreList': [90.26331329345703], 'NoiseList': [99.47721099853516], 'BlurList': [99.99376678466797], 'FnfList': [100.0], 'GlassList': [99.98408508300781], 'MaskList': [86.59868621826172], 'IlluList': [99.99360656738281], 'PoseList': [7.566176891326904]}, 'DenseFeatureLength': 1024, 'Pupils': [94.2857437133789, 161.79794311523438, 4.954795837402344, 152.37928771972656, 155.88494873046875, 4.954795837402344], 'GenderList': [0], 'PoseList': [-17.756900787353516, -6.129109859466553, -6.328291893005371], 'Masks': [0], 'DenseFeatures': [-0.013079792261123657, 0.03860211372375488, -0.03327830508351326, -0.004178051371127367, 0.03764724358916283, 0.05017240345478058, -0.02594659850001335, -0.047795359045267105, 0.011404608376324177, 0.03609126806259155, 0.007491060998290777, -0.025500880554318428, 0.0015673731686547399, 0.045232806354761124, -0.0281417835503816

In [4]:
Data=response["Data"]
Data

{'BeautyList': [89],
 'FaceRectangles': [75, 88, 133, 173],
 'Qualities': {'ScoreList': [90.26331329345703],
  'NoiseList': [99.47721099853516],
  'BlurList': [99.99376678466797],
  'FnfList': [100.0],
  'GlassList': [99.98408508300781],
  'MaskList': [86.59868621826172],
  'IlluList': [99.99360656738281],
  'PoseList': [7.566176891326904]},
 'DenseFeatureLength': 1024,
 'Pupils': [94.2857437133789,
  161.79794311523438,
  4.954795837402344,
  152.37928771972656,
  155.88494873046875,
  4.954795837402344],
 'GenderList': [0],
 'PoseList': [-17.756900787353516, -6.129109859466553, -6.328291893005371],
 'Masks': [0],
 'DenseFeatures': [-0.013079792261123657,
  0.03860211372375488,
  -0.03327830508351326,
  -0.004178051371127367,
  0.03764724358916283,
  0.05017240345478058,
  -0.02594659850001335,
  -0.047795359045267105,
  0.011404608376324177,
  0.03609126806259155,
  0.007491060998290777,
  -0.025500880554318428,
  0.0015673731686547399,
  0.045232806354761124,
  -0.02814178355038166,

In [ ]:


"""

名称		示例值	描述



Pupils	[243.02786254882812,149.0238037109375,7.3701672554016113,315.86138916015625,140.451416015625,7.3701672554016113]	
左右两个瞳孔的中心点坐标和半径，每个人脸6个浮点数，顺序为[left_iris_cenpt.x, left_iris_cenpt.y, left_iris_radius, right_iris_cenpt.x, right_iris_cenpt.y, right_iris_radis]。

GenderList	
识别到的人脸性别。如有多个人脸，则依次返回。

0：女性
1：男性

Expressions	
人脸识别的表情结果。包括：

0：中性
1：微笑

DenseFeatures		
人脸识别返回的特征。如有多个人脸，则依次返回。

FaceCount	
检测出来的人脸个数。

Landmarks	
人脸的特征点定位结果，每个人脸返回一组特征点位置，
表示方式为（x0, y0, x1, y1, ……）。如有多个人脸，则依次返回，返回定位浮点数。

LandmarkCount	
人脸检测的特征点数目，目前固定为105点。依次为：眉毛24点，眼睛32点，鼻子6点，嘴巴34点，外轮廓9点。

Qualities	
人脸质量情况，分数越高表示越有利于识别。


BeautyList	
颜值分数，分值越高颜值越高，取值范围(0-100]。

HatList	
人脸是否佩戴帽子。
    0：无帽子
    1：有帽子
    
FaceProbabilityList	
返回人脸检测的概率，取值范围0~1。如有多个人脸，则依次返回。例如有两个人脸则返回[face_prob1, face_prob2]。

Glasses	
人脸是否佩戴眼镜。
        0：不戴眼镜
        1：佩戴普通眼镜
        2：佩戴墨镜
        
FaceRectangles	
返回人脸矩形框，格式为[left, top, width, height]。如有多个人脸，则依次顺延，返回矩形框。例如有两个人脸则返回[left1, top1, width1, height1, left2, top2, width2, height2]。

PoseList	
返回人脸姿态，格式为[yaw, pitch, roll]。如有多个人脸，则依次返回。

yaw为左右角度，取值范围-90~90。
pitch为上下角度，取值范围-90~90。
roll为平面旋转角度，取值范围-180~180。

AgeList	
图片中人脸年龄，取值范围0~100。如有多个人脸，依次返回多个人脸的年龄。

DenseFeatureLength	
人脸识别返回的特征维度，目前固定为1024。

Masks	
人脸是否佩戴口罩。

0：未佩戴口罩
1：佩戴口罩
2：未正确佩戴口罩
"""